In [15]:
import numpy as np
import math
from scipy.stats import norm
from matplotlib import pyplot as plt
from numpy import linalg as LA
from scipy.stats import unitary_group
import random
import scipy.integrate as spi

In [16]:
sigma_space = np.logspace(-1,1,100)
thresh_space = np.linspace(-5,5,100)
M =1
mu = 0
sigma_teta = (1/math.sqrt(2))
rho_q = 1
rho_a = 1
sim = pow(10,2)
########Teta
real_teta = np.random.normal(mu, sigma_teta,M)
im_teta = np.random.normal(mu, sigma_teta,M)
teta = real_teta + 1j*im_teta
teta = teta.reshape(M,1)

In [17]:
def Matrix(na,nq): #new model
    H_mat = np.zeros((na*M,M), complex)
    G_mat = np.zeros((nq*M,M), complex)
    for i in range(0,na*M,M):
        if M>1:
            H_mat[i:M+i,:] = math.sqrt(rho_a)*unitary_group.rvs(M)
        else:
            x1 = random.random()
            y1 = math.sqrt(1 - pow(x1, 2))
            H_mat[i:M + i, :] = math.sqrt(rho_a)*(x1+1j*y1)
    for i in range(0,nq*M,M):
        if M > 1:
            G_mat[i:M+i,:] = math.sqrt(rho_q)*unitary_group.rvs(M)
        else:
            x2 = random.random()
            y2 = np.sqrt(1 - np.power(x2, 2))
            G_mat[i:M + i, :] = math.sqrt(rho_a)*(x2 + 1j * y2)
    return H_mat, G_mat

In [18]:
def thresh_G(n_q, Mat):
    if M>1:
        G_teta=Mat@((mu+1j*mu)*np.ones(M))
    else:
        G_teta=Mat*((mu+1j*mu)*np.ones(M))
    return G_teta.real.reshape(M*n_q, 1), G_teta.imag.reshape(M*n_q, 1)

In [19]:
def x(sigma, n_a,n_q, matrix): #the observations- function of teta
    sigma_w_a = sigma * (1 / math.sqrt(2))
    real_w_a = np.random.normal(mu, sigma_w_a, M*n_a)
    im_w_a = np.random.normal(mu, sigma_w_a, M*n_a)
    w_a = real_w_a + 1j * im_w_a
    w_a = w_a.reshape(M*n_a, 1)

    sigma_w_q = sigma * (1 / math.sqrt(2))
    real_w_q = np.random.normal(mu, sigma_w_q, M*n_q)
    im_w_q = np.random.normal(mu, sigma_w_q, M*n_q)
    w_q = real_w_q + 1j * im_w_q
    w_q = w_q.reshape(M*n_q, 1)

    if M>1:
        x_a = matrix[0]@teta + w_a
        y = matrix[1]@teta + w_q
    else:
        x_a = matrix[0]*teta + w_a
        y = matrix[1]*teta + w_q

    x_q = (1 / math.sqrt(2)) * (np.sign(y.real - (thresh_G(n_q,matrix[1])[0]) +
                            1j * np.sign(y.imag - ((thresh_G(n_q,matrix[1])[1])))))
    return x_a.reshape(M*n_a, ), x_q.reshape(M*n_q, )

def samp(sigma, n_a,n_q, matrix, observ): #samples
    sigma_teta_samp = (1/math.sqrt(2))
    real_teta_samp = np.random.normal(mu, sigma_teta_samp, (M,observ))
    im_teta_samp = np.random.normal(mu, sigma_teta_samp, (M,observ))
    teta_samp = real_teta_samp + 1j*im_teta_samp

    sigma_w_a_samp = sigma * (1 / math.sqrt(2))
    real_w_a_samp = np.random.normal(mu, sigma_w_a_samp, (M*n_a,observ))
    im_w_a_samp = np.random.normal(mu, sigma_w_a_samp, (M*n_a,observ))
    w_a_samp = real_w_a_samp + 1j * im_w_a_samp

    sigma_w_q_samp = sigma * (1 / math.sqrt(2))
    real_w_q_samp = np.random.normal(mu, sigma_w_q_samp,(M*n_q,observ))
    im_w_q_samp = np.random.normal(mu, sigma_w_q_samp,(M*n_q,observ))
    w_q_samp = real_w_q_samp + 1j * im_w_q_samp

    x_a_samp = (matrix[0]@teta_samp)+w_a_samp
    y_samp = (matrix[1]@teta_samp) + w_q_samp

    x_q_samp = (1 / math.sqrt(2)) * (np.sign(y_samp.real - (thresh_G(n_q,matrix[1])[0]) +
                                        1j * np.sign(y_samp.imag - ((thresh_G(n_q, matrix[1])[1])))))
    return x_a_samp, x_q_samp, teta_samp.reshape(M,observ)

def samp_teta(observ): #samples-for CRB function (d_k)
    real_teta_samp = np.random.normal(mu, sigma_teta, (M, observ))
    im_teta_samp = np.random.normal(mu, sigma_teta, (M, observ))
    teta_samp = real_teta_samp + 1j * im_teta_samp
    return  teta_samp.reshape(M,observ)

def covariance(v1,v2):
    normv1 = np.mean(v1,1)
    normv2 = np.mean(v2,1)
    v = v1-normv1.reshape(np.shape(v1)[0],1)
    u = v2 -normv2.reshape(np.shape(v2)[0],1)
    result = [v[:,i].reshape(np.shape(v)[0], 1)@u[:,i].transpose().reshape(1, np.shape(u)[0]) for i in range(np.shape(v)[1])]
    return np.mean(result,0)

In [58]:
def MSE_zertothresh_analytic(sigma, n_a,n_q):
    alpha = (2 / math.pi) * math.acos(rho_q / (rho_q + pow(sigma, 2)))
    beta = (1-alpha)/rho_q
    first = (rho_a*n_a)/(rho_a*n_a+pow(sigma, 2))
    second = (2*rho_q*n_q*pow(sigma,4))/(math.pi*(rho_q+pow(sigma, 2))*(alpha+beta*rho_q*n_q)*pow(rho_a*n_a+pow(sigma, 2),2))
    return math.sqrt(M)*(1-first-second) #Frobenius  norm


In [59]:
def CRB(sigma, n_a, n_q,matrix, observ):
    teta_samp = samp_teta(observ)
    g_teta = matrix[1] @ teta_samp

    zeta_real = (math.sqrt(2) / sigma * (g_teta.real - thresh_G(n_q, matrix[1])[0]))
    zeta_im = (math.sqrt(2) / sigma * (g_teta.imag - thresh_G(n_q, matrix[1])[1]))

    pdf_real = norm.pdf(zeta_real)
    cdf_real = norm.cdf(zeta_real)
    pdf_im = norm.pdf(zeta_im)
    cdf_im = norm.cdf(zeta_im)

    d_vec = np.divide(np.power(pdf_real, 2), np.multiply(cdf_real, (norm.cdf(-zeta_real)))) + \
            np.divide(np.power(pdf_im, 2), np.multiply(cdf_im, (norm.cdf(-zeta_im))))
    d = np.mean(d_vec, axis=1)
    Fisher_q = (1 /(2 * pow(sigma, 2)))*(matrix[1].transpose().conjugate() @ np.diag(d) @ matrix[1])

    Fisher_a = rho_a*n_a/pow(sigma,2)
    J = 1+Fisher_a+Fisher_q
    return float(1/J.real)

In [60]:
def CRB_M_levels(sigma,matrix, observ,tao_vec, n): #M=1
    tao_vec = np.insert(tao_vec, 0, -10000)
    tao_vec = np.append(tao_vec, 10000)
    teta_samp = samp_teta(observ)
    g_teta =(matrix[1]@teta_samp)[0]

    zeta = [math.sqrt(2)/(sigma*(tao_vec[i]+n-g_teta.real)) for i in range(len(tao_vec))]
    the_sum = np.sum([((rho_q/2*pow(sigma,2))*norm.pdf(zeta[i+1])-norm.pdf(zeta[i]))/(norm.cdf(zeta[i+1])-norm.cdf(zeta[i])) for  i in range(len(zeta)-1)],axis=0)
    Fisher_q = np.mean(the_sum, axis=0)

    Fisher_a = rho_a*n_a/pow(sigma,2) #(1/pow(sigma,2))*(matrix[0].transpose().conjugate()@matrix[0])
    J = 1+Fisher_a+Fisher_q
    return float(1/J.real)

In [61]:
n_a = [1,1,2,2]
n_q = [100,100,40,100]
tao_list = [0] #M levels
matrix_const1 = Matrix(n_a[0],n_q[0])

L_Estimator_analytic1 = [MSE_zertothresh_analytic(sigma_space[i], n_a[0],n_q[0]) for i in range(len(sigma_space))]
CRB = [CRB(sigma_space[i],n_a[0],n_q[0],matrix_const1,sim) for i in range(len(sigma_space))]
#CRB_M = [CRB_M_levels(sigma_space[i],matrix_const1, sim*100, tao_list, 0) for i in range(len(thresh_space))]

plt.plot(10*np.log10(1/sigma_space), L_Estimator_analytic1, color="green", label='n_a={}, n_q = {}, analytic'.format(n_a[0], n_q[0]))
plt.plot(10*np.log10(1/sigma_space), CRB, color="red", label='n_a={}, n_q = {}, CRB'.format(n_a[0], n_q[0]))
#plt.plot(10*np.log10(1/sigma_space), CRB_M, color="blue", label='n_a={}, n_q = {}, M levels'.format(n_a[0], n_q[0]))

plt.title("M={}, mu={}, variance ={}".format(M,mu,2*pow(sigma_teta,2)))
plt.yscale('log')
plt.xlabel("SNR [dB]")
plt.legend()
plt.legend()
plt.show()

TypeError: only size-1 arrays can be converted to Python scalars